In [ ]:
%reload_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [ ]:
import cdsapi
import geopandas as gpd
import pandas as pd
import plotly.express as px
import numpy as np
from dotenv import load_dotenv
from ecowater_data_research import loaders, transformers
import datetime as dt

load_dotenv()

In [ ]:
var = "total_precipitation"
year = 2022
month = 1
day = 1
date = dt.datetime(year, month, day)
request = {
    "variable": var,
    "year": year,
    "month": month,
    "day": day,
    "time": [
        "00:00",
        "01:00",
        "02:00",
        "03:00",
        "04:00",
        "05:00",
        "06:00",
        "07:00",
        "08:00",
        "09:00",
        "10:00",
        "11:00",
        "12:00",
        "13:00",
        "14:00",
        "15:00",
        "16:00",
        "17:00",
        "18:00",
        "19:00",
        "20:00",
        "21:00",
        "22:00",
        "23:00",
    ],
    "area": [
        52,
        -5,
        40,
        10,
    ],
    "format": "netcdf",
}
target = f"../data/copernicus/{var}_{date.strftime('%Y%m%d')}.nc"
name = "reanalysis-era5-land"

In [ ]:
c = cdsapi.Client()
c.retrieve(
    name=name,
    request=request,
    target=target,
)

In [ ]:
df = loaders.load_from_cds(target)
gdf = transformers.transform_to_gdf(df)
gdf = gdf[~gdf["tp"].isna()]

In [ ]:
gdf = gdf
with open("../data/mapping/contours_lambert_93.json") as file:
    regions_table = gpd.read_file(file)
    regions_table["geometry"].crs = "EPSG:2154"
    regions_table["geometry"] = regions_table["geometry"].to_crs(epsg=4326)
    regions_table: gpd.GeoDataFrame = regions_table[regions_table["geometry"].is_valid]
    result = None
    for i, row in regions_table.iterrows():
        print(i / regions_table.reset_index().shape[0], end="\r")
        gdf_in = gdf.geometry.within(row.geometry)
        if not gdf_in.any():
            continue
        is_in = gdf_in.values
        gdf_in[is_in] = row.id
        gdf_in[~is_in] = np.nan
        if result is None:
            result = gdf_in
        else:
            result[result.isna()] = gdf_in[result.isna()]
    result.name = "id"
    concat = pd.concat([gdf, result], axis=1)
    concat = concat[~concat.id.isna()]
    transformed_gdf = (
        concat[["id", "time", "tp"]].groupby(["id", "time"]).sum().reset_index()
    )
    df = pd.merge(transformed_gdf, regions_table, on="id")
    df = df.set_index("id")
    del regions_table
    del result
    del transformed_gdf

In [ ]:
fig = px.choropleth(
    df,
    geojson=gpd.GeoSeries(df.geometry),
    locations=df.index,
    color="tp",
    color_continuous_scale="Inferno",
)
fig.update_geos(fitbounds="locations", visible=False)

In [ ]:
fig